In [1]:
import warnings
warnings.filterwarnings('ignore')
from IPython.core.display import display, HTML
display(HTML("<style>.container {width: 100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import pyodbc 
import sqlalchemy
from sqlalchemy.engine import URL
import numpy as np
from datetime import datetime
from sqlOps import sqlDBManagement
from dotenv import load_dotenv
import os
from pipelineAlerts import Email
pd.set_option('display.max_columns', None)
load_dotenv()
OLTP_SERVER = os.getenv("SERVER")
OLTP_USER = os.getenv("USER")
OLTP_PASS = os.getenv("PASS")
OLTP_DB = os.getenv("DB")

server = 'SV2021\SQLVIETINSOFT' 
database = 'Paradise_BLH' 
username = 'vts.sa' 
password = 'LuaThieng1@3' 


In [7]:
sql = sqlDBManagement(server, username, password, database, dbms="mssql")

In [8]:
def get_new_hires():
    query = '''
    select "ChangedDate" from tblEmployeeStatusHistory as s 
    where EmployeeStatusID = 0 and not exists (
        select 1 from tblEmployeeStatusHistory as h where h.EmployeeID = s.EmployeeID and h.ChangedDate < s.ChangedDate
    )
    '''
    df_new_hire = sql.getDataFramebyQuery(query)
    df_new_hire["New Hire"] = 1  
    df_new_hire['Month'] = df_new_hire['ChangedDate'].dt.month_name().str.slice(stop=3)
    df_new_hire['Year'] = df_new_hire['ChangedDate'].dt.year
    df_new_hire = df_new_hire.drop('ChangedDate', axis = 1)
    df_new_hire = df_new_hire.groupby(['Month', 'Year'], as_index= False).agg(
                                    NewHire = pd.NamedAgg('New Hire', aggfunc='sum'))
    return df_new_hire

In [9]:
def get_emp_history():
    att_dates = set([str(date).split(' ')[0] for date in EmpAtt['AttDate']])
    working_emp_list = []
    for date in att_dates:
        qry = f'''
        DECLARE @FromDate datetime, @ToDate datetime
        -- from day to day: is the period of the month
        SET @FromDate ='{date}'
        SET @ToDate ='{date}'

        select COUNT(EmployeeCodeReal) AS working_emp from dbo.fn_vtblEmployeeList_Bydate(@ToDate,'-1',null) 
        WHERE TerminateDate is null or TerminateDate > @FromDate
        '''
        working_emp = sql.getDataFramebyQuery(qry)
        working_emp_list.append(working_emp["working_emp"][0])
        
    working_emp = pd.DataFrame(zip(att_dates,working_emp_list), columns=['att_dates', 'workingEmp'])
    working_emp['att_dates'] = working_emp['att_dates'].astype('datetime64[ns]')
    
    statusDim = sql.getDataFramebyQuery("SELECT EmployeeStatusID,EmployeeStatusEN  FROM tblEmployeeStatus")
    empStatus = sql.getDataFramebyQuery("select EmployeeID, EmployeeStatusID, ChangedDate from tblEmployeeStatusHistory")
    empStatus = pd.merge(empStatus, statusDim, on="EmployeeStatusID")
    
    EmpHistory = pd.merge(working_emp, empStatus, left_on="att_dates", right_on="ChangedDate")
    EmpHistory = EmpHistory.drop(columns=['att_dates', 'EmployeeStatusID'])
    EmpHistory = EmpHistory.drop(EmpHistory[EmpHistory['EmployeeStatusEN'] == 'Working'].index)

    EmpHistory['Terminated']=[1 if i=='Terminated' else 0 for i in EmpHistory['EmployeeStatusEN']]
    EmpHistory['Maternity']=[1 if i=='Maternity' else 0 for i in EmpHistory['EmployeeStatusEN']]
    EmpHistory['Before maternity']=[1 if i=='Before maternity' else 0 for i in EmpHistory['EmployeeStatusEN']]

    EmpHistory = EmpHistory.drop(columns=['EmployeeStatusEN'])

    EmpHistory['month_num'] = EmpHistory['ChangedDate'].dt.month
    EmpHistory['Month'] = EmpHistory['ChangedDate'].dt.month_name().str.slice(stop=3)
    EmpHistory['Year'] = EmpHistory['ChangedDate'].dt.year

    EmpHistory = EmpHistory.drop(columns=['EmployeeID', 'ChangedDate'])
    
    EmpHistory = EmpHistory.groupby(['Month', 'Year'], as_index= False).agg(
                                Working = pd.NamedAgg('workingEmp', aggfunc=pd.Series.mode),
                                Terminated = pd.NamedAgg('Terminated', aggfunc='sum'),
                                Maternity = pd.NamedAgg('Maternity', aggfunc='sum'),
                                BeforeMaternity = pd.NamedAgg('Before maternity', aggfunc='sum'),
                                month_num = pd.NamedAgg('month_num', aggfunc=pd.Series.mode)
            )
    EmpHistory['Working'] = EmpHistory['Working'].apply(lambda x : x[0] if ((str(x).startswith("[")) & (str(x).endswith("]"))) else x )
    return EmpHistory

In [10]:
EmpHistory = get_emp_history()
new_hires = get_new_hires()
MainEmpHistory = pd.merge(EmpHistory, new_hires, on=['Month', 'Year'])

NameError: name 'EmpAtt' is not defined

## MainEmpStatus

In [5]:
EmpInfo = sql.getDataFramebyQuery("SELECT * FROM Emp_Info;")
EmpStatus = sql.getDataFramebyQuery("SELECT * FROM EmpStatus;")
Empdim = sql.getDataFramebyQuery("SELECT * FROM EmpDim;")
EmpInfoEmpdim = EmpInfo.merge(Empdim, on= 'EmployeeID')
MainEmpStatus = pd.merge(EmpInfoEmpdim, EmpStatus,  how='left', left_on=['EmployeeID','EmployeeStatusEN'], right_on = ['EmployeeID','EmployeeStatusEN'])

MainEmpStatus = MainEmpStatus[['EmployeeID', 'FullName_x', 'MobilePhone', 'Email', 'Birthday','BirthPlace', 'NativeCountry', 'PhotoImage',\
                    'HireDate','SexNameEN', 'ID_Number', 'ID_Issue_Date', 'EthnicNameEN', 'DistrictNameEN', 'ResidentAdd',\
                   'ProvinceNameEN','WardNameEN','EducationalBase', 'SocialNo','EmployeeStatusEN','DivisionNameEN','DepartmentNameEN',\
                   'PositionNameEN','MaritalStatusNameEN', 'ChangedDate']]
MainEmpStatus

,EmployeeID,FullName_x,MobilePhone,Email,Birthday,BirthPlace,NativeCountry,PhotoImage,HireDate,SexNameEN,ID_Number,ID_Issue_Date,EthnicNameEN,DistrictNameEN,ResidentAdd,ProvinceNameEN,WardNameEN,EducationalBase,SocialNo,EmployeeStatusEN,DivisionNameEN,DepartmentNameEN,PositionNameEN,MaritalStatusNameEN,ChangedDate
0,0001,1,None,None,NaT,None,None,None,2021-05-18,Male,None,NaT,None,None,None,None,None,None,None,Terminated,Head Ofiice,New employee,Operator,None,2021-05-18
1,0002,2,None,None,2021-05-09,None,None,None,2020-03-06,Male,None,NaT,None,None,None,None,None,None,None,Terminated,Head Ofiice,New employee,Operator,None,2020-03-06
2,0003,0003,None,None,NaT,None,None,None,2020-09-16,Male,None,NaT,None,None,None,None,None,None,None,Terminated,Head Ofiice,New employee,Operator,None,2020-09-29
3,0035,DA LAN,None,None,NaT,None,None,None,2021-05-18,Male,None,NaT,None,None,None,None,None,None,None,Terminated,Head Ofiice,New employee,Operator,None,2021-05-18
4,0044,VAN,None,None,NaT,None,None,None,2021-05-18,Male,None,NaT,None,None,None,None,None,None,None,Terminated,Head Ofiice,New employee,Operator,None,2021-05-18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2238,4197,NGUYỄN THỊ NGA,0378929369,None,1996-06-13,Tỉnh Đắk Lắk,None,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,2021-05-11,Female,241621672,2013-07-25,Kinh,Ea H'leo District,Thôn 6,Dak Lak,Ea Ral Ward,None,6622357995,Working,BLUE HORIZON,OFFICE,HR Assistant,Single,2021-05-11
2239,4329,HOÀNG THỊ TUYẾT NGA,0962024840,None,1998-10-14,Tỉnh Kon Tum,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,2021-11-08,Female,245382060,2016-07-11,Tày,Dak R'Lap District,None,Dak Nong,Hung Binh Ward,None,7416320994,Working,BLUE HORIZON,AUTOMATION,Stitcher,Married,2021-11-08
2240,4605,PHẠM THỊ NGỌC NỮ,0336164064,None,2004-03-04,None,None,b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,2022-05-23,Female,352772037,2020-03-09,Kinh,Cho Moi District,Kiến Thuận 2,An Giang,Kien Thanh Ward,None,None,Terminated,BLUE HORIZON,LINE 7,Helper,Single,2022-10-20
2241,3506,LÊ THỊ LỢI,0348577897,None,1989-02-22,Tỉnh Nghệ An,"Quỳnh Văn, Quỳnh Lưu, Nghệ An",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,2019-08-06,Female,186653084,2005-07-21,Kinh,Thu Duc District,"41/6/6A, Đường 11, Khu phố 2",Ho Chi Minh City,Truong Tho Ward,5/12,7911216658,Working,BLUE HORIZON,LINE 3,Stitcher,Single,2019-08-06


In [6]:
MainEmpStatus.drop_duplicates(subset=['EmployeeID'], keep='last', inplace=True)
#Main_Emp_EmpStatus[Main_Emp_EmpStatus.duplicated(['EmployeeID'])]
print(MainEmpStatus.shape)
MainEmpStatus[MainEmpStatus.EmployeeID == "3767"]

(2214, 25)


,EmployeeID,FullName_x,MobilePhone,Email,Birthday,BirthPlace,NativeCountry,PhotoImage,HireDate,SexNameEN,ID_Number,ID_Issue_Date,EthnicNameEN,DistrictNameEN,ResidentAdd,ProvinceNameEN,WardNameEN,EducationalBase,SocialNo,EmployeeStatusEN,DivisionNameEN,DepartmentNameEN,PositionNameEN,MaritalStatusNameEN,ChangedDate
1082,3767,Shahid Muhammad Saad,None,None,1995-10-14,None,Pakistan,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,2020-02-22,Male,NL5142243,2019-12-26,None,None,None,None,None,None,None,Terminated,BLUE HORIZON,FOREIGN,A&P Assistant,Single,2022-08-23


In [7]:
MainEmpStatus.describe(include='all').T

,count,unique,top,freq,first,last
EmployeeID,2214,2214,0001,1,NaT,NaT
FullName_x,2214,1692,LINH,18,NaT,NaT
MobilePhone,912,826,0909472320,4,NaT,NaT
Email,2,2,hr@bhgloves.com,1,NaT,NaT
Birthday,1053,786,1995-01-01 00:00:00,14,1957-01-01,2022-05-30
BirthPlace,764,41,Tỉnh An Giang,328,NaT,NaT
NativeCountry,587,367,"Hà Bao 2, Đa Phước, An Phú, An Giang",94,NaT,NaT
PhotoImage,896,895,b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,2,NaT,NaT
HireDate,2214,564,2021-05-18,801,NaT,NaT
SexNameEN,2214,2,Male,1548,NaT,NaT


In [8]:
resg_lst = []
for index, row in MainEmpStatus[['EmployeeStatusEN', 'ChangedDate']].iterrows():
    if row[0] == 'Terminated':
        resg_lst.append(row[1])
    else:
        resg_lst.append(np.nan)
MainEmpStatus['date_of_resg'] = resg_lst

from datetime import datetime
current_date = datetime.today().strftime('%Y-%m-%d')
current_date = pd.to_datetime(current_date)
MainEmpStatus['HireDate'] = pd.to_datetime(MainEmpStatus['HireDate'])
exp_in_comp = []
for index , row in MainEmpStatus[['date_of_resg','HireDate']].iterrows():
    if pd.isna(row[0]):
        exp_in_comp.append(((current_date- row[1])/pd.Timedelta(1, unit='d')))
    else:
        exp_in_comp.append(np.nan)
MainEmpStatus['exp_in_comp'] = exp_in_comp

MainEmpStatus['work_duration'] = (MainEmpStatus['date_of_resg'] - MainEmpStatus['HireDate'])/pd.Timedelta(1, unit='d')
age =  round(((current_date -  MainEmpStatus['Birthday'])/pd.Timedelta(1,unit='d'))/360, 0)
MainEmpStatus['Age'] = age
len(MainEmpStatus)

2214

In [9]:
MainEmpStatus[MainEmpStatus.duplicated(['EmployeeID'])]

,EmployeeID,FullName_x,MobilePhone,Email,Birthday,BirthPlace,NativeCountry,PhotoImage,HireDate,SexNameEN,ID_Number,ID_Issue_Date,EthnicNameEN,DistrictNameEN,ResidentAdd,ProvinceNameEN,WardNameEN,EducationalBase,SocialNo,EmployeeStatusEN,DivisionNameEN,DepartmentNameEN,PositionNameEN,MaritalStatusNameEN,ChangedDate,date_of_resg,exp_in_comp,work_duration,Age


In [10]:
MainEmpStatus.isna().sum()

EmployeeID                0
FullName_x                0
MobilePhone            1302
Email                  2212
Birthday               1161
BirthPlace             1450
NativeCountry          1627
PhotoImage             1318
HireDate                  0
SexNameEN                 0
ID_Number              1193
ID_Issue_Date          1210
EthnicNameEN           1200
DistrictNameEN         1295
ResidentAdd            1386
ProvinceNameEN         1279
WardNameEN             1332
EducationalBase        2052
SocialNo               1544
EmployeeStatusEN          2
DivisionNameEN            0
DepartmentNameEN          0
PositionNameEN            0
MaritalStatusNameEN    1380
ChangedDate               0
date_of_resg            619
exp_in_comp            1595
work_duration           619
Age                    1161
dtype: int64

In [11]:
MainEmpStatus = MainEmpStatus.rename(columns={'FullName_x':'FullName', 'SexNameEN':'Gender', 'EthnicNameEN':'EthnicName',
                                                        'EducationalBase':'Education', 'DivisionNameEN':'Division', 'DepartmentNameEN':'Department',
                                                        'PositionNameEN': 'Position', 'EmployeeStatusEN': 'EmployeeStatus',
                                                        'WardNameEN': 'WardName', 'MaritalStatusNameEN':'MaritalStatus', 'ProvinceNameEN': 'ProvinceName'})
MainEmpStatus.columns

Index(['EmployeeID', 'FullName', 'MobilePhone', 'Email', 'Birthday',
       'BirthPlace', 'NativeCountry', 'PhotoImage', 'HireDate', 'Gender',
       'ID_Number', 'ID_Issue_Date', 'EthnicName', 'DistrictNameEN',
       'ResidentAdd', 'ProvinceName', 'WardName', 'Education', 'SocialNo',
       'EmployeeStatus', 'Division', 'Department', 'Position', 'MaritalStatus',
       'ChangedDate', 'date_of_resg', 'exp_in_comp', 'work_duration', 'Age'],
      dtype='object')

In [12]:
MainEmpStatus[MainEmpStatus.EmployeeStatus=='Working']['EmployeeStatus'].count() #Total Working

593

In [13]:
MainEmpStatus[MainEmpStatus.duplicated(subset=['EmployeeID'])]

,EmployeeID,FullName,MobilePhone,Email,Birthday,BirthPlace,NativeCountry,PhotoImage,HireDate,Gender,ID_Number,ID_Issue_Date,EthnicName,DistrictNameEN,ResidentAdd,ProvinceName,WardName,Education,SocialNo,EmployeeStatus,Division,Department,Position,MaritalStatus,ChangedDate,date_of_resg,exp_in_comp,work_duration,Age


In [14]:
MainEmpStatus['EmployeeStatus'].value_counts()

Terminated          1595
Working              593
Maternity             12
After maternity        7
Before maternity       5
Name: EmployeeStatus, dtype: int64

### Transactions/Facts

In [15]:
EmpAtt = sql.getDataFramebyQuery('''
                 SELECT EmployeeID, AttDate, AttStart, AttEnd, WorkingTime , DATENAME(dw, AttDate) AS "Day Of The Week"
                 FROM tblHasTA
                 WHERE DATENAME(dw, AttDate) != 'Sunday'
                 ORDER BY AttDate DESC;
                 ''')
EmpAtt

,EmployeeID,AttDate,AttStart,AttEnd,WorkingTime,Day Of The Week
0,1009,2022-11-08,2022-11-08 07:18:34,NaT,NaN,Tuesday
1,1017,2022-11-08,2022-11-08 07:30:00,2022-11-08 16:30:00,8.0,Tuesday
2,1030,2022-11-08,2022-11-08 07:20:58,NaT,NaN,Tuesday
3,1042,2022-11-08,2022-11-08 06:48:54,NaT,NaN,Tuesday
4,1043,2022-11-08,2022-11-08 07:17:56,NaT,NaN,Tuesday
...,...,...,...,...,...,...
401210,13880,2000-01-01,NaT,NaT,NaN,Saturday
401211,14384,2000-01-01,NaT,NaT,NaN,Saturday
401212,0048,2000-01-01,NaT,NaT,NaN,Saturday
401213,12342,2000-01-01,NaT,NaT,NaN,Saturday


In [117]:
attToday = EmpAtt[EmpAtt['AttDate']=='2022-10-26']
attToday

,EmployeeID,AttDate,AttStart,AttEnd,WorkingTime,Day Of The Week
2865,988,2022-10-26,2022-10-26 07:17:55,2022-10-26 20:30:19,8.0,Wednesday
2866,985,2022-10-26,2022-10-26 07:15:58,2022-10-26 16:34:11,8.0,Wednesday
2867,974,2022-10-26,2022-10-26 06:54:37,2022-10-26 16:34:53,8.0,Wednesday
2868,894,2022-10-26,2022-10-26 07:08:06,2022-10-26 16:32:45,8.0,Wednesday
2869,873,2022-10-26,2022-10-26 07:21:33,2022-10-26 16:34:02,8.0,Wednesday
...,...,...,...,...,...,...
3438,1043,2022-10-26,NaT,NaT,NaN,Wednesday
3439,1042,2022-10-26,2022-10-26 06:38:06,2022-10-26 16:36:33,8.0,Wednesday
3440,1030,2022-10-26,2022-10-26 07:19:38,2022-10-26 16:30:55,8.0,Wednesday
3441,1017,2022-10-26,2022-10-26 07:25:01,2022-10-26 16:35:01,8.0,Wednesday


In [118]:
todayAtt = EmpAtt[EmpAtt['AttDate'] == '2022-10-26']
todayAtt.describe(include='object').T

,count,unique,top,freq
EmployeeID,578,578,988,1
Day Of The Week,578,1,Wednesday,578


In [119]:
todayAtt.describe(include=np.number).T

,count,mean,std,min,25%,50%,75%,max
WorkingTime,453.0,7.997792,0.046984,7.0,8.0,8.0,8.0,8.0


In [120]:
absentToday = todayAtt.AttStart.isna().sum()
absentToday

118

In [121]:
presentToday = todayAtt.AttStart.notnull().sum()
presentToday

460

In [122]:
uniqueEmps = todayAtt.EmployeeID.nunique()
uniqueEmps

578

In [123]:
countEmp = todayAtt.EmployeeID.count()
countEmp

578

In [124]:
absentToday = countEmp - presentToday
absentToday

118

In [125]:
EmpStatusSummary = MainEmpStatus['EmployeeStatus'].value_counts()
EmpStatusSummary

Terminated          1590
Working              594
Maternity             14
After maternity        6
Before maternity       5
Name: EmployeeStatus, dtype: int64

In [126]:
terminated = EmpStatusSummary[EmpStatusSummary.index == 'Terminated']
terminated[0]

1590

In [127]:
main_df = pd.merge(EmpAtt, MainEmpStatus, how="left", on='EmployeeID')
main_df

,EmployeeID,AttDate,AttStart,AttEnd,WorkingTime,Day Of The Week,FullName,MobilePhone,Email,Birthday,BirthPlace,NativeCountry,PhotoImage,HireDate,Gender,ID_Number,ID_Issue_Date,EthnicName,DistrictNameEN,ResidentAdd,ProvinceName,WardName,Education,SocialNo,EmployeeStatus,Division,Department,Position,ChangedDate,date_of_resg,exp_in_comp,work_duration,Age
0,1009,2022-11-01,2022-11-01 07:13:16,NaT,NaN,Tuesday,RO VI DAH,0961504167,None,1992-07-05,Tỉnh An Giang,"Hà Bao 2, Đa Phước, An Phú, An Giang",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,2011-10-10,Female,352108219,2008-04-07,Chăm,An Phu District,Hà Bao 2,An Giang,Da Phuoc Ward,None,7413007969,Working,BLUE HORIZON,LINE 7,Stitcher,2011-10-10,NaT,4040.0,NaN,31.0
1,1017,2022-11-01,NaT,NaT,0.0,Tuesday,NGUYỄN THỊ HỒNG HẠNH,0973747169,None,1984-05-10,None,"Phường Phú Thọ, TDM, Bình DƯơng",b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\...,2008-05-15,Female,281242564,2015-09-29,Kinh,Thu Dau Mot City,None,Binh Duong,Phu Tho Ward,Đại học,7408044768,Working,BLUE HORIZON,OFFICE,Manager,2022-10-05,NaT,5283.0,NaN,39.0
2,1030,2022-11-01,2022-11-01 07:15:59,NaT,NaN,Tuesday,NGUYỄN THỊ KIM TRINH,0922777655,None,1968-08-09,Tỉnh Bình Dương,"Bình Phước A, Bình Chuẩn, Thuận An, Bình Dương",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,2011-11-19,Female,280412023,2009-08-04,Kinh,Thuan An Town,23/4 Bình Phước A,Binh Duong,Binh Chuan Ward,None,7412056713,Working,BLUE HORIZON,LINE 5,Helper,2011-11-19,NaT,4000.0,NaN,55.0
3,1042,2022-11-01,2022-11-01 06:38:05,NaT,NaN,Tuesday,VÕ THỊ HUYỀN TRANG,0967824676,None,1990-12-20,Tỉnh Bình Dương,"Bình Nhâm, Thuận An, Bình Dương",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,2009-03-07,Female,280965984,2006-08-01,Kinh,Thuan An Town,61A Bình Hòa,Binh Duong,Binh Nham Ward,None,7409104650,Working,BLUE HORIZON,LINE 5,Stitcher,2009-03-07,NaT,4987.0,NaN,32.0
4,1043,2022-11-01,2022-11-01 07:23:52,NaT,NaN,Tuesday,THỊ BÍCH,0385916815,None,1975-01-01,Tỉnh Kiên Giang,"Đông Thạnh, An Minh, Kiên Giang",b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01...,2010-02-25,Female,370768075,2009-07-26,Kinh,An Minh District,None,Kien Giang,Dong Thanh Ward,1/12,7410066510,Working,BLUE HORIZON,LINE 1,Helper,2010-02-25,NaT,4632.0,NaN,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397762,13880,2000-01-01,NaT,NaT,NaN,Saturday,13880,None,None,NaT,None,None,None,2000-01-01,Male,None,NaT,None,None,None,None,None,None,None,Working,Head Ofiice,New employee,Operator,2000-01-01,NaT,8340.0,NaN,NaN
397763,14384,2000-01-01,NaT,NaT,NaN,Saturday,14384,None,None,NaT,None,None,None,2000-01-01,Male,None,NaT,None,None,None,None,None,None,None,Working,Head Ofiice,New employee,Operator,2000-01-01,NaT,8340.0,NaN,NaN
397764,0048,2000-01-01,NaT,NaT,NaN,Saturday,0048,None,None,NaT,None,None,None,2000-01-01,Male,None,NaT,None,None,None,None,None,None,None,Working,Head Ofiice,New employee,Operator,2000-01-01,NaT,8340.0,NaN,NaN
397765,12342,2000-01-01,NaT,NaT,NaN,Saturday,12342,None,None,NaT,None,None,None,2000-01-01,Male,None,NaT,None,None,None,None,None,None,None,Working,Head Ofiice,New employee,Operator,2000-01-01,NaT,8340.0,NaN,NaN


In [128]:
# To calculate the number of hiring every month (WIP)
#MainEmpStatus['hiring_MonthDate'] = np.where(pd.notnull(MainEmpStatus['HireDate'], )

In [317]:
import pandas as pd
import numpy as np
import pyodbc 
import sqlalchemy
from sqlalchemy.engine import URL
import numpy as np
from datetime import datetime
from sqlOps import sqlDBManagement
from dotenv import load_dotenv
import os
from pipelineAlerts import Email
load_dotenv()
OLTP_SERVER = os.getenv("SERVER")
OLTP_USER = os.getenv("USER")
OLTP_PASS = os.getenv("PASS")
OLTP_DB = os.getenv("DB")

server = 'SV2021\SQLVIETINSOFT' 
database = 'Paradise_BLH' 
username = 'vts.sa' 
password = 'LuaThieng1@3' 

class HRtoDWH:
    def __init__(self):
        self.oltp = sqlDBManagement(server, username, password, database, dbms="mssql")
        self.olap = sqlDBManagement(OLTP_SERVER, OLTP_USER, OLTP_PASS, OLTP_DB)
    def get_MainEmpStatus(self):
        
        EmpInfo = self.oltp.getDataFramebyQuery("SELECT * FROM Emp_Info;")
        EmpStatus = self.oltp.getDataFramebyQuery("SELECT * FROM EmpStatus;")
        Empdim = self.oltp.getDataFramebyQuery("SELECT * FROM EmpDim;")
        EmpInfoEmpdim = EmpInfo.merge(Empdim, on= 'EmployeeID')
        MainEmpStatus = pd.merge(EmpInfoEmpdim, EmpStatus,  how='left', left_on=['EmployeeID','EmployeeStatusEN'], right_on = ['EmployeeID','EmployeeStatusEN'])

        MainEmpStatus = MainEmpStatus[['EmployeeID', 'FullName_x', 'MobilePhone', 'Email', 'Birthday','BirthPlace', 'NativeCountry', 'PhotoImage',\
                            'HireDate','SexNameEN', 'ID_Number', 'ID_Issue_Date', 'EthnicNameEN', 'DistrictNameEN', 'ResidentAdd',\
                           'ProvinceNameEN','WardNameEN','EducationalBase', 'SocialNo','EmployeeStatusEN','DivisionNameEN','DepartmentNameEN',\
                           'PositionNameEN','ChangedDate']]
        MainEmpStatus.drop_duplicates(subset=['EmployeeID'], keep='last', inplace=True)
        resg_lst = []
        for index, row in MainEmpStatus[['EmployeeStatusEN', 'ChangedDate']].iterrows():
            if row[0] == 'Terminated':
                resg_lst.append(row[1])
            else:
                resg_lst.append(np.nan)
        MainEmpStatus['date_of_resg'] = resg_lst

        from datetime import datetime
        current_date = datetime.today().strftime('%Y-%m-%d')
        current_date = pd.to_datetime(current_date)
        MainEmpStatus['HireDate'] = pd.to_datetime(MainEmpStatus['HireDate'])
        exp_in_comp = []
        for index , row in MainEmpStatus[['date_of_resg','HireDate']].iterrows():
            if pd.isna(row[0]):
                exp_in_comp.append(((current_date- row[1])/pd.Timedelta(1, unit='d')))
            else:
                exp_in_comp.append(np.nan)
        MainEmpStatus['exp_in_comp'] = exp_in_comp

        MainEmpStatus['work_duration'] = (MainEmpStatus['date_of_resg'] - MainEmpStatus['HireDate'])/pd.Timedelta(1, unit='d')
        age =  round(((current_date -  MainEmpStatus['Birthday'])/pd.Timedelta(1,unit='d'))/360, 0)
        MainEmpStatus['Age'] = age
        self.MainEmpStatus = MainEmpStatus.rename(columns={'FullName_x':'FullName', 'SexNameEN':'Gender', 'EthnicNameEN':'EthnicName',
                                                        'EducationalBase':'Education', 'DivisionNameEN':'Division', 'DepartmentNameEN':'Department',
                                                        'PositionNameEN': 'Position', 'EmployeeStatusEN': 'EmployeeStatus',
                                                        'WardNameEN': 'WardName', 'ProvinceNameEN': 'ProvinceName'})
    def get_EmpAtt(self):
        self.EmpAtt = self.oltp.getDataFramebyQuery('''
                 SELECT EmployeeID, AttDate, AttStart, AttEnd, WorkingTime , DATENAME(dw, AttDate) AS "Day Of The Week"
                 FROM tblHasTA
                 WHERE DATENAME(dw, AttDate) != 'Sunday'
                 ORDER BY AttDate DESC;
                 ''')
    
    def __inset_into_remote(self, df, table_name):
        id_count = len(df['EmployeeID'])
        print(id_count)
        id_count_OLAP = self.olap.getDataFramebyQuery(f'''SELECT COUNT(EmployeeID) from {table_name} WHERE EmployeeID;''')
        print(id_count_OLAP)
        id_count_OLAP = id_count_OLAP["COUNT(EmployeeID)"][0]
        print(id_count_OLAP)
        
        if id_count_OLAP == id_count:
            pass 
        else:
            new_record_count = id_count - id_count_OLAP
            new_records = df.tail(new_record_count)
            if new_records.empty:
                print("No new records, skipping insertion...")
                
            else:
                try:
                    print(f"Inserting {new_record_count} new records in {table_name}...")
                    self.olap.saveDataFrameIntoDB(new_records, table_name)
                    
                except Exception as e:
                    print(f'Something went while inserting the data in {table_name}')
                    raise e
    def run_pipeline(self):
        try:
            self.get_MainEmpStatus()
            self.get_EmpAtt()  
            self.__inset_into_remote(self.MainEmpStatus, "MainEmpStatus")
            self.__inset_into_remote(self.EmpAtt, "EmpAtt")
            print("******* Job completed sccessfully! *******")
        except:
            alert = Email()
            alert.pipeline_failure_alert()

In [318]:
dataJob = HRtoDWH()

In [320]:
dataJob.olap.executeOperation("DELETE FROM MainEmpHistory")

In [ ]:
dataJob = HRtoDWH()
dataJob.run_pipeline()